In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

from helpers.data_generation.file_management import read_hdf5
from helpers.data_generation.error_generation import Residual, CombineDataset
from helpers.model.helpers_model import NeuralNet

import warnings
warnings.filterwarnings('ignore') 

In [2]:
ratio = 0.75
percent = np.array([0.005, 0.015, 0.005])
size = 6000

batch_size = 50
max_epoch = 150

opti_name = np.array(['SGD', 'SGD/momentum', 'Adam'])



res = Residual()
res.build(size, ratio = ratio, per_error = percent)

str_ID =  "S"+str(size)+"R"+str(int(ratio*100))
[final_array, metadata] = read_hdf5(str_ID)
metadata ['ID'] = np.arange(0,final_array.shape[0])

In [ ]:
plt.figure()
for norm in np.array([True, False]):
    data_set = CombineDataset(metadata,'ID','class',final_array,norm)

    data_train, data_test = train_test_split(data_set,train_size=0.9,random_state=42)


    loader_train = DataLoader(data_train, batch_size = batch_size, 
                            num_workers = 0, drop_last=True)

    loader_test = DataLoader(data_test, batch_size = batch_size, 
                            num_workers = 0, drop_last=True)


    for opti in opti_name:
        print('----------------------------------------------------------------------------------')

        test_SGDnorm = np.zeros(max_epoch)
        netbasic = NeuralNet('BasicCNN', opti)
        for epoch in range(max_epoch):
            netbasic.train(loader_train)
            res = netbasic.test(loader_test, verbose = False)
            test_SGDnorm[epoch] = res
        
        if norm:
            txt = "Finished Training- Normalized: "+ opti +" - epoch: {:.3f} - auc: {:.3f} \n" 
            print(txt.format(netbasic.opti_epoch, netbasic.max_met))
            plt.plot(np.arange(1, max_epoch+1), test_SGDnorm, label='Norm '+opti)
        else:
            txt = "Finished Training- Not normalized: "+ opti +" - epoch: {:.3f} - auc: {:.3f} \n" 
            print(txt.format(netbasic.opti_epoch, netbasic.max_met))
            plt.plot(np.arange(1, max_epoch+1), test_SGDnorm, label=opti)
plt.legend()
plt.grid()
plt.savefig('figures/optimizers.jpeg')
plt.show()

----------------------------------------------------------------------------------
Finished Training- Normalized: SGD - epoch: 150.000 - auc: 0.892 

----------------------------------------------------------------------------------
Finished Training- Normalized: SGD/momentum - epoch: 36.000 - auc: 0.917 

----------------------------------------------------------------------------------
Finished Training- Normalized: Adam - epoch: 49.000 - auc: 0.905 

----------------------------------------------------------------------------------
Finished Training- Not normalized: SGD - epoch: 136.000 - auc: 0.898 

----------------------------------------------------------------------------------
Finished Training- Not normalized: SGD/momentum - epoch: 129.000 - auc: 0.916 

----------------------------------------------------------------------------------
